# Простая сетевая модель и метрики центральности её узлов

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import networkx as nx
from math import ceil
from tqdm.notebook import tqdm
import networkx as nx
import glob
import json
import pandas as pd
import seaborn as sns
from ipywidgets import interact
import graph_tool.all as gt

%matplotlib inline

In [2]:
# files = glob.glob('../data/raw_data/vk-api-saved/*.json')

# print('Number of crawled user pages: {}'.format(len(files)))

# groups_list_df = []
# walls_list_df = []
# singleLeaders_list_df = []
# problematique = []
# raw = []

# for fpath in tqdm(files):
#     with open(fpath) as f:
#         content = json.load(f)
        
#         if 'execute_errors' not in content:
#             errors = [dict(method='wall.get', code='No Error'),
#                       dict(method='groups.get', code='No Error'),
#                       dict(method='execute.singleLeader', code='No Error')]
#         else:
#             errors = content['execute_errors']
        
#         groups = dict(id=content['response'][0][0]['id'], response=content['response'][1], 
#                       error=[e for e in errors if e['method']=='groups.get'])
#         wall = dict(id=content['response'][0][0]['id'], response=content['response'][2], 
#                             error=[e for e in errors if e['method']=='groups.get'])
        
#         groups_list_df.append(groups)
#         walls_list_df.append(wall)
# 
# def drop_errors(df):
#     df = df[df['error'].apply(lambda x : x[0]['error_code'] in [15, 29] 
#                               if (len(x)>0 and 'error_code' in x[0]) else True)].copy(deep=True)
#     return df

# groups_df = drop_errors(pd.DataFrame(groups_list_df))
# walls_df = drop_errors(pd.DataFrame(walls_list_df))

__________________________

#### Построим сеть дружеских связей. 

In [25]:
friendship_graph = nx.read_edgelist('../data/raw_data/friends.csv', delimiter=',')

In [26]:
num_introverts = sum([d==1 for n,d in tqdm(friendship_graph.degree())])

In [28]:
num_introverts/friendship_graph.number_of_edges()

0.3182279724385737

In [8]:
density = (2*friendship_graph.number_of_edges())/(friendship_graph.number_of_nodes()**2)

In [12]:
print('Number of edges: {}'.format(friendship_graph.number_of_edges()))
print('Number of vertices: {}'.format(friendship_graph.number_of_nodes()))
print('Graph density: {}'.format(round(density, 8)))

Number of edges: 19650507
Number of vertices: 9275663
Graph density: 4.6e-07


In [14]:
max_connected_component = friendship_graph.subgraph(max(nx.connected_components(friendship_graph), key=lambda x : len(x)))

In [21]:
del friendship_graph

In [22]:
cc_nodes = max_connected_component.number_of_nodes()
cc_edges = max_connected_component.number_of_edges()

In [24]:
cc_density = (2*cc_edges)/(cc_nodes**2)
print('Number of edges: {}'.format(cc_edges))
print('Number of vertices: {}'.format(cc_nodes))
print('Graph density: {}'.format(round(cc_density, 8)))

Number of edges: 19650506
Number of vertices: 9275661
Graph density: 4.6e-07


In [5]:
# %%time
# friendship = gt.load_graph_from_csv('../data/raw_data/friends.csv')

In [4]:
%%time
pagerank = gt.pagerank(friendship)

CPU times: user 1min 59s, sys: 280 ms, total: 2min
Wall time: 16.5 s


In [31]:
pagerank_arr = pagerank.get_array()

In [37]:
with open('../data/pagerank_centralities.csv', 'w', encoding='utf-8') as f:
    pagerank_arr.tofile(f, sep=',')

In [ ]:
%%time
betweenness = gt.betweenness(friendship)

In [6]:
%%time
hits = gt.hits(friendship)

CPU times: user 5min 41s, sys: 568 ms, total: 5min 41s
Wall time: 45.3 s


In [12]:
a = hits[-1].get_array()

In [19]:
nonzero_hits = np.argwhere(a)

In [21]:
len(nonzero_hits)

9275661

In [23]:
friendship.num_edges()

39301013

In [24]:
friendship.num_vertices()

9275663

In [38]:
with open('../data/hits_centralities.csv', 'w', encoding='utf-8') as f:
    hits[-1].get_array().tofile(f, sep=',')